In [4]:
# general
import numpy as np
import pandas as pd
from IPython.display import display
import importlib

# first used in exercise one
from sklearn.metrics.pairwise import linear_kernel, polynomial_kernel

In [3]:
x_simple = np.array([3,2,0,1,-1,-2]).reshape(3,2)
x_simple

array([[ 3,  2],
       [ 0,  1],
       [-1, -2]])

We can calculate the linear kernel manually or via function:

In [26]:
x_simple.dot(x_simple.T)

array([[13,  2, -7],
       [ 2,  1, -2],
       [-7, -2,  5]])

In [24]:
linear_kernel(x_simple)

array([[ 13.,   2.,  -7.],
       [  2.,   1.,  -2.],
       [ -7.,  -2.,   5.]])

And we can calculate the polynomial kernel with c = 1 and d = 2 manually or via function (note for the polynomial_kernel function we need to pass gamma = 1 to override the default that normalizes the calculated value by 1/n):

In [29]:
(x_simple.dot(x_simple.T) + 1)**2

array([[196,   9,  36],
       [  9,   4,   1],
       [ 36,   1,  36]])

In [27]:
polynomial_kernel(x_simple, degree=2, coef0=1, gamma=1)

array([[ 196.,    9.,   36.],
       [   9.,    4.,    1.],
       [  36.,    1.,   36.]])

_Write a function kerneleval that computes, for any set of datapoints x1; : : : ; xn and a
new datapoint x?, the vector of kernel evaluations [k(x1; x?); : : : ; k(xn; x?)]T ._

Note that here we're not saying to use the kernel matrix at all, I think - the reference to 'kernel evaluation' means to use the kernel _function_, not the kernel matrix. I think this is just the predict step, which - for a linear kernel - is the sum of the inner products between each observation and the value to be predicted, weighted by each learned value of alpha.